# 中文情感分析模型训练教程 (Chinese Sentiment Analysis Tutorial)

欢迎来到这个通过 **Hugging Face Transformers** 和 **BERT** 进行中文情感分析的交互式教程！

**本教程专为学习设计，所有的核心代码都直接包含在下方的单元格中，无需跳转查看外部文件。**

我们将一起完成以下步骤：
1. **环境检查**：确认 PyTorch 和 MPS (Mac 加速) 是否可用。
2. **配置 (Configuration)**：定义模型参数、路径和训练设置。
3. **数据预处理**：从 Hugging Face 加载数据，清洗并进行 Tokenization (分词)。
4. **数据可视化**：绘制数据分布图，了解正负样本比例。
5. **模型构建与训练**：自动检测是否已有训练好的模型，如果有则直接加载，无需等待。
6. **评估与预测**：查看训练曲线（如果刚训练完）并亲手进行测试。

---

## 1. 导入库与环境检查
我们首先导入必要的 Python 库，并检查您的 Mac 是否支持硬件加速。

In [ ]:
import os
import sys
import json
import torch
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 检查 Mac MPS 加速
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ 成功开启 Mac MPS (Metal Performance Shaders) 硬件加速！")
else:
    device = torch.device("cpu")
    print("⚠️ 未检测到 MPS，将使用 CPU 训练 (速度较慢)。")

## 2. 定义配置 (Config)
我们将所有的参数都在这里定义好，方便集中管理。

In [ ]:
class Config:
    # === 模型配置 ===
    BASE_MODEL = "google-bert/bert-base-chinese"
    NUM_LABELS = 3
    MAX_LENGTH = 128
    
    # === 路径配置 ===
    # 我们把 Notebook 训练的模型保存在这里的 checkpoints 文件夹下
    OUTPUT_DIR = "../checkpoints/tutorial_model"
    LOG_DIR = "../results/notebook_logs"
    
    # === 训练参数 ===
    BATCH_SIZE = 32
    LEARNING_RATE = 2e-5
    NUM_EPOCHS = 3
    
    # === 标签映射 ===
    LABEL2ID = {'negative': 0, 'neutral': 1, 'positive': 2}
    ID2LABEL = {0: 'negative', 1: 'neutral', 2: 'positive'}
    
print("配置已加载。")

## 3. 数据处理 (Data Processor)
加载数据，清洗，统一格式。

In [ ]:
class DataProcessor:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def clean_data(self, example):
        text = example['text']
        if text is None or "此用户未填写评价内容" in text or len(text.strip()) < 2:
            return False
        return True

    def unify_labels(self, example):
        label = example['label']
        if isinstance(label, str):
            label = label.lower()
            if label in ['negative', 'pos', '0']: return {'label': 0}
            elif label in ['neutral', 'neu', '1']: return {'label': 1}
            elif label in ['positive', 'neg', '2']: return {'label': 2}
        return {'label': int(label)}

    def tokenize_function(self, examples):
        return self.tokenizer(
            examples['text'],
            padding="max_length",
            truncation=True,
            max_length=Config.MAX_LENGTH
        )

    def get_processed_dataset(self):
        print("正在加载数据集...")
        # 使用本地 data 目录缓存
        cache_dir = "../data"
        ds_clap = load_dataset("clapAI/MultiLingualSentiment", split="train[:3000]", trust_remote_code=True, cache_dir=cache_dir)
        ds_med = load_dataset("OpenModels/Chinese-Herbal-Medicine-Sentiment", split="train[:3000]", trust_remote_code=True, cache_dir=cache_dir)
        
        if 'review_text' in ds_med.column_names: ds_med = ds_med.rename_column('review_text', 'text')
        if 'sentiment_label' in ds_med.column_names: ds_med = ds_med.rename_column('sentiment_label', 'label')
        if 'language' in ds_clap.column_names: ds_clap = ds_clap.filter(lambda x: x['language'] == 'zh')
            
        common_cols = ['text', 'label']
        combined = concatenate_datasets([ds_clap.select_columns(common_cols), ds_med.select_columns(common_cols)])
        
        combined = combined.filter(self.clean_data)
        combined = combined.map(self.unify_labels)
        
        print("正在分词...")
        tokenized_ds = combined.map(self.tokenize_function, batched=True)
        return tokenized_ds.train_test_split(test_size=0.1)

# 初始化
tokenizer = AutoTokenizer.from_pretrained(Config.BASE_MODEL)
dataset = None 
# 注意：为了节省时间，只有在需要训练时才加载数据
print("Tokenizer 准备就绪。")

## 5. 智能训练逻辑
这里是核心修改点：**如果检测到模型已经存在，就直接跳过训练，直接加载！**

In [ ]:
# 检查是否存在已保存的模型
model_path = Config.OUTPUT_DIR
should_train = True

if os.path.exists(model_path) and os.path.exists(os.path.join(model_path, "config.json")):
    print(f"✅ 检测到已训练的模型位于: {model_path}")
    print("🚀 将跳过训练过程，直接加载该模型！")
    should_train = False
    
    # 直接加载微调后的模型
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    model.to(device)
    
else:
    print("⚠️ 未检测到已存在的模型，准备开始训练...")
    
    # 1. 既然要训练，那我们就需要加载数据了
    if dataset is None:
        processor = DataProcessor(tokenizer)
        dataset = processor.get_processed_dataset()
        
    # 2. 加载基座模型
    model = AutoModelForSequenceClassification.from_pretrained(
        Config.BASE_MODEL, 
        num_labels=Config.NUM_LABELS,
        id2label=Config.ID2LABEL,
        label2id=Config.LABEL2ID
    )
    model.to(device)

    # 3. 定义指标
    def compute_metrics(pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
        acc = accuracy_score(labels, preds)
        return {'accuracy': acc, 'f1': f1}

    # 4. 训练参数
    training_args = TrainingArguments(
        output_dir=Config.OUTPUT_DIR,
        num_train_epochs=Config.NUM_EPOCHS,
        per_device_train_batch_size=16,
        logging_dir=Config.LOG_DIR,
        logging_steps=10,
        eval_strategy="steps",
        eval_steps=100,
        save_steps=100,
        save_total_limit=2,
    )

    # 5. Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        processing_class=tokenizer,
        compute_metrics=compute_metrics,
    )

    # 6. 开始训练
    print("开始训练...")
    trainer.train()
    
    # 7. 保存
    print(f"保存模型到: {Config.OUTPUT_DIR}")
    trainer.save_model(Config.OUTPUT_DIR)
    tokenizer.save_pretrained(Config.OUTPUT_DIR)

## 6. 模型推理 (Interactive Inference)
无论你是刚训练完，还是直接加载的模型，都可以直接用下面的代码来测试！

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
        pred_idx = torch.argmax(probs).item()
        return Config.ID2LABEL[pred_idx], probs[0][pred_idx].item()

text_input = widgets.Text(placeholder='输入中文...', description='评论:')
btn = widgets.Button(description="分析", button_style='info')
out = widgets.Output()

def on_click(b):
    with out:
        out.clear_output()
        if not text_input.value: return
        label, conf = predict(text_input.value)
        print(f"结果: {label} (置信度: {conf:.4f})")

btn.on_click(on_click)
display(text_input, btn, out)